In [1]:
from sys import path
path.append("../")

In [2]:
import jax
import jax.numpy as jnp
from flax.struct import dataclass 

from abc import abstractmethod

In [3]:
from fg import Gaussian

# Inter Robot

In [4]:
x = jnp.array([
    [1,1],
    [2,2],
    [3,3],
    [4,4]
])
states = jnp.stack((x, x + 0.1, x + 0.3))
vels = jnp.zeros_like(states)
states = jnp.concat((states, vels), axis=-1)
states, states.shape

(Array([[[1. , 1. , 0. , 0. ],
         [2. , 2. , 0. , 0. ],
         [3. , 3. , 0. , 0. ],
         [4. , 4. , 0. , 0. ]],
 
        [[1.1, 1.1, 0. , 0. ],
         [2.1, 2.1, 0. , 0. ],
         [3.1, 3.1, 0. , 0. ],
         [4.1, 4.1, 0. , 0. ]],
 
        [[1.3, 1.3, 0. , 0. ],
         [2.3, 2.3, 0. , 0. ],
         [3.3, 3.3, 0. , 0. ],
         [4.3, 4.3, 0. , 0. ]]], dtype=float32, weak_type=True),
 (3, 4, 4))

In [5]:
@dataclass
class InterRobotMapping:
    points: jnp.ndarray
    other_robot: int

@dataclass
class InterRobotVar2FacMessages:
    robot: jnp.ndarray
    other_robot: jnp.ndarray

@dataclass
class InterRobotFac2VarMessages:
    robot: jnp.ndarray
    other_robot: jnp.ndarray

(3, 4, 4)

In [31]:
def find_closest_robot(states: jnp.ndarray):
    def find_closest_robot_across_horizon(robot, other_robots):
        closest_index = jnp.argmin(jnp.linalg.norm(robot[0:2] - other_robots[:,0:2], axis=1))
        return closest_index

    def find_batched_closest_robot(batch_states, i):
        modified_states = batch_states.at[i].set(jnp.inf)
        return jax.vmap(find_closest_robot_across_horizon, in_axes=(0, 1))(batch_states[i], modified_states)
    return jax.vmap(find_batched_closest_robot, in_axes=(None, 0))(states, jnp.arange(states.shape[0]))
closest_robots = find_closest_robot(states) # ideally, should be # (N - 1, 4, 2)
closest_robots

Array([[1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1]], dtype=int32)

In [36]:
N_STATES = 4
POSE_NOISE = 1e-15
DYNAMICS_NOISE = 0.005
OBSTACLE_NOISE = 0.005
INTER_ROBOT_NOISE = 0.005

class Factor:
    def __init__(
        self, state: jnp.array, state_precision: jnp.ndarray, dims: jnp.array, linear: bool = True
    ) -> None:
        self._state = state
        self._state_precision = state_precision
        self._linear = linear
        self._dims = dims

    def calculate_likelihood(self) -> Gaussian:
        return Gaussian(
            self._calc_info(self._state, self._state_precision),
            self._calc_precision(self._state, self._state_precision),
            self._dims
        )

    @abstractmethod
    def _calc_measurement(self, state: jnp.ndarray) -> jnp.ndarray:
        pass

    def _calc_info(self, state: jnp.ndarray, precision: jnp.ndarray) -> jnp.ndarray:
        X = state
        if self._linear:
            eta = precision @ (jnp.zeros(N_STATES) - self._calc_measurement(state))
        else:
            J = jax.jacfwd(self._calc_measurement)(state)
            eta = (J.T @ precision) @ (
                (J @ X.reshape((-1,1))) + 0 - self._calc_measurement(state).reshape((-1,1))
            )
        return eta.squeeze()

    def _calc_precision(self, state: jnp.ndarray, precision: jnp.ndarray) -> jnp.ndarray:
        if self._linear:
            return precision
        else:
            J = jax.jacfwd(self._calc_measurement)(state)
            return J.T @ precision @ J

class InterRobotFactor(Factor):
    def __init__(
        self,
        state: jnp.ndarray,
        agent_radius: float,
        critical_distance: float,
        t: jnp.ndarray, #ndarray just to hold time,
        dims: jnp.ndarray,
    ) -> None:
        self._critical_distance = critical_distance
        self._agent_radius = agent_radius
        precision = jnp.pow(t * INTER_ROBOT_NOISE, -2) * jnp.eye(N_STATES)
        super(InterRobotFactor, self).__init__(state, precision, dims, False)

    def _calc_measurement(self, state: jnp.ndarray):
        current_state = state[0:4]
        other_state = state[4:]
        dist = self._calc_dist(current_state, other_state)
        measurement = jax.lax.select(
            dist < self._critical_distance, jnp.full((4,), 1.0 - dist / self._critical_distance), jnp.zeros((4,)) 
        )
        return measurement
    
    def _calc_dist(self, state: jnp.array, other_state: jnp.array):
        return jnp.linalg.norm(state[0:2] - other_state[0:2])

ir_factor =  InterRobotFactor(jnp.array([-0.1, 0., 0.0, 0.0, 0.1, 0.0, 0.0, 0.0]), 0.2, 1.5, jnp.array([1.]), jnp.array([1., 1., 1., 1., 100., 100., 100., 100.]))
g = ir_factor.calculate_likelihood()
g = g.marginalize(jnp.array([1., 1., 1., 1.]))
g

Gaussian(info=Array([nan, nan, nan, nan], dtype=float32), precision=Array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]], dtype=float32), dims=Array([100., 100., 100., 100.], dtype=float32))

In [17]:
def init_inter_robot_var2fac_msgs() -> InterRobotVar2FacMessages:
    n_agents = 2
    time_horizon = 4
    dummy = jnp.zeros((n_agents, time_horizon))
    robot_msgs = jax.vmap(jax.vmap(lambda _, var: Gaussian.identity(var)))(dummy, jnp.repeat(jnp.arange(1, time_horizon + 1)[jnp.newaxis,:], n_agents, axis=0))
    other_robot_msgs = jax.vmap(jax.vmap(lambda _, var: Gaussian.identity(var)))(dummy, jnp.repeat(jnp.full((1, time_horizon), 100.), n_agents, axis=0))
    return InterRobotVar2FacMessages(robot_msgs, other_robot_msgs) # (N, K, )

msgs = init_inter_robot_var2fac_msgs()

In [29]:
msgs.robot.dims[0][0]

Array([1, 1, 1, 1], dtype=int32)

# Obstacle

In [9]:
obstacle = jnp.array([
    [4.2, 4.2],
    [1.0, 1.0]
])

def find_closest_obstacle(states, obstacles):
    def find_closest_obstacle_for_pt(state_t, obstacles):
        closest_obstacle_idx = jnp.argmin(jnp.linalg.norm(state_t - obstacles, axis=1))
        return obstacles[closest_obstacle_idx]
    def batch_find_closest_obstacle(agent_states, obstacles):
        return jax.vmap(find_closest_obstacle_for_pt, in_axes=(0, None))(agent_states, obstacles)
    return jax.vmap(batch_find_closest_obstacle, in_axes=(0, None))(states, obstacles)
closest_obstacle = find_closest_obstacle(states, obstacle)
closest_obstacle, closest_obstacle.shape

(Array([[[1. , 1. ],
         [1. , 1. ],
         [4.2, 4.2],
         [4.2, 4.2]],
 
        [[1. , 1. ],
         [1. , 1. ],
         [4.2, 4.2],
         [4.2, 4.2]],
 
        [[1. , 1. ],
         [1. , 1. ],
         [4.2, 4.2],
         [4.2, 4.2]]], dtype=float32),
 (3, 4, 2))

# Random

In [40]:
x = jnp.array([[1,2,3,4,5,6], [7,8,9,10,11,12]])
y = x * 2
x + y

Array([[ 3,  6,  9, 12, 15, 18],
       [21, 24, 27, 30, 33, 36]], dtype=int32)

In [42]:
x = jnp.arange(32).reshape((8,4))
x[1:-1:2] + x[2:-1:2]

Array([[12, 14, 16, 18],
       [28, 30, 32, 34],
       [44, 46, 48, 50]], dtype=int32)

In [43]:
x[1:-1:2], x[2:-1:2]

(Array([[ 4,  5,  6,  7],
        [12, 13, 14, 15],
        [20, 21, 22, 23]], dtype=int32),
 Array([[ 8,  9, 10, 11],
        [16, 17, 18, 19],
        [24, 25, 26, 27]], dtype=int32))